In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import re
from transformers import BertTokenizer, BertTokenizerFast
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertModel
import torch.optim as optim
import numpy as np
from tqdm import tqdm

In [ ]:
class BERT_LSTM_GRU(nn.Module):
    def __init__(self, bert_model, hidden_dim, embedding_dim):
        super(BERT_LSTM_GRU, self).__init__()
        self.bert = bert_model
        self.bert.requires_grad_(False)
        self.lstm = nn.LSTM(768, hidden_dim, batch_first=True).to(torch.float32)
        self.gru = nn.GRU(hidden_dim, hidden_dim, batch_first=True).to(torch.float32)
        self.lm_head = nn.Linear(hidden_dim, embedding_dim).to(torch.float32)

    def forward(self, x):
        with torch.no_grad():
            embedding = self.bert(x).last_hidden_state
        x, _ = self.lstm(embedding)
        x, _ = self.gru(x)
        x = self.lm_head(x)
        return x

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained('bert-base-uncased')
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = BERT_LSTM_GRU(bert_model, 512, tokenizer.vocab_size)
checkpoint = torch.load("../models/model_epoch2_step20000.pt", map_location="cpu")

In [ ]:
model.load_state_dict(checkpoint["model_state_dict"])

In [ ]:
print(checkpoint["loss"])

In [ ]:
def generate_text(model, tokenizer, prompt, max_length=50, device=device):
    model.eval()

    tokens = tokenizer.encode(prompt, add_special_tokens=True, return_tensors='pt').to(device)

    generated_tokens = tokens.clone()

    with torch.no_grad():
        for _ in range(max_length):
            logits = model(generated_tokens)

            next_token_logits = logits[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)

            generated_tokens = torch.cat([generated_tokens, next_token], dim=1)

            if next_token.item() == tokenizer.eos_token_id:
                break

    generated_text = tokenizer.decode(generated_tokens.squeeze(0), skip_special_tokens=True)

    return generated_text

In [ ]:
prompt = "Once upon a time, there was a big dog named Max. Max had a red collar that he wore every day. He loved to play and run in the park with his friends."
print(generate_text(model, tokenizer, prompt))

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
generate_text(model, tokenizer, prompt)

In [ ]:
from nltk.translate.meteor_score import meteor_score

reference = [prompt.lower().split()]
candidate = generate_text(model, tokenizer, prompt).split()

meteor_score(reference, candidate)

In [ ]:
from rouge_score import rouge_scorer

reference = prompt
candidate = generate_text(model, tokenizer, prompt)

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

scores = scorer.score(reference, candidate)

print(scores)